In [1]:
#libraries
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
import torch.nn as nn

In [64]:
#constants that can be fine-tuned
batch_size = 64
hidden_layer_nodes = 32
learning_rate = 0.001

In [36]:
def accuracy(predictions, actuals):
  num_correct = 0
  for i in range(len(predictions)):
    pred = torch.argmax(predictions[i])
    if pred == actuals[i]:
      num_correct += 1

  accuracy = num_correct / len(predictions)
  return accuracy

In [65]:
# retrieve data and initialize
df = pd.read_csv('sample_data/mnist_train_small.csv')
df_test = pd.read_csv('sample_data/mnist_test.csv')

x_train_df = df.iloc[:, 1:]
y_train_df = df.iloc[:, 0]
x_test_df = df_test.iloc[:, 1:]
y_test_df = df_test.iloc[:, 0]

x_train = torch.tensor(x_train_df.values, dtype=torch.float32)
y_train = torch.tensor(y_train_df.values, dtype=torch.long) # no need to one hot encode, CrossEntropyLoss does it for us by providing correct index
x_test = torch.tensor(x_test_df.values, dtype=torch.float32)
y_test = torch.tensor(y_test_df.values, dtype=torch.long)

num_features = len(x_train[0])
num_classes = len(torch.unique(y_train))

train_dataset = TensorDataset(x_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [66]:
# neural network 1
def network_1(d) -> nn.Module:
  model = nn.Sequential(
      nn.Linear(d, hidden_layer_nodes),
      nn.ReLU(),
      nn.Linear(hidden_layer_nodes, hidden_layer_nodes),
      nn.ReLU(),
      nn.Linear(hidden_layer_nodes, num_classes)
  )

  return model

In [67]:
model = network_1(num_features)
loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [70]:
model.train()
train_accuracy = 0
while train_accuracy < 0.99:
  for X, y in train_loader:
    predictions = model.forward(X)
    train_accuracy = accuracy(predictions, y)

    output = loss(predictions, y)

    # backprop
    optimizer.zero_grad()
    output.backward()
    optimizer.step()

In [71]:
test_predictions = model.forward(x_test)
test_accuracy = accuracy(test_predictions, y_test)
print("Test accuracy:", test_accuracy)

Test accuracy: 0.9394939493949395
